In [1]:
with open('imports.py', 'r') as file:
    imports_code = file.read()

with open('utils.py', 'r') as file:
    utils_code = file.read()

with open('funcoes.py', 'r') as file:
    funcoes_code = file.read()

exec(imports_code)
exec(utils_code)
exec(funcoes_code)

# Extrair dados de posicao completos

In [2]:
# Criando uma sessao
session = requests.Session()

# Fazendo várias requisições com a mesma sessão
dados_corredor = get_dados(session, base_url, categoria="Corredor", token=token)
dados_empresa = get_dados(session, base_url, categoria="Empresa", token=token)
dados_posicao = get_dados(session, base_url, categoria="Posicao", token=token)

# Finalizando a sessão quando não for mais necessária
session.close()

# Gerar variável para salvar no nome do arquivo
agora = datetime.now().strftime("%Y-%m-%d %H%M%S")

# Salvando dados
path_posicao = f"raw\posicao\{agora}.json"
path_corredor = f"raw\corredor\{agora}.json"
path_empresa = f"raw\empresa\{agora}.json"

salvar_json(dados_corredor, path_corredor, 'utf-8')
salvar_json(dados_empresa, path_empresa, 'utf-8')
salvar_json(dados_posicao, path_posicao, 'utf-8')


Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!


'Arquivo salvo com sucesso!'

In [3]:
# Extrair a lista de todas as linhas da posicao
lista_linhas = list(set([i['cl'] for i in dados_posicao['l']]))
print(len(lista_linhas))

1977


# Extrair dados de previsao de chegada de cada linha retornada no json de posição

In [4]:
# Lista que vai receber as requisições de previsao de cada linha retornada no metódo de posição 
dados_previsao = []

for i in lista_linhas[:100]:
    previsao_linha = get_dados(session, base_url, categoria="Previsao", metodo="Linha", sufix="codigoLinha", parametro=str(i), token=token)
    
    # Adiciona o código da linha 'i' ao JSON de previsão retornado
    if previsao_linha:  # Verifica se a resposta não é nula
        previsao_linha['codigo_linha'] = i  # Adiciona o código da linha ao JSON

        # Adiciona a previsão à lista
        dados_previsao.append(previsao_linha)

# Salvando dados de previsao de chegada por linha
agora = datetime.now().strftime("%Y-%m-%d %H%M%S")
path_previsao = f"raw\previsao\{agora}.json"
salvar_json(dados_previsao, path_previsao, 'utf-8')


# Finalizando a sessão quando não for mais necessária
session.close()
    

Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com s

# Extrair dados de paradas de cada linha retornada no json de posição

In [5]:
# Lista que vai receber as requisições de parada de cada linha retornada no metódo de posição 
dados_paradas = []

for i in lista_linhas[:100]:
    parada_linha = get_dados(session, base_url, categoria="Parada", metodo="BuscarParadasPorLinha", sufix="codigoLinha", parametro=str(i), token=token)
    
    # Adiciona o código da linha em cada parada retornada
    if parada_linha:  # Verifica se há dados de paradas
        for parada in parada_linha:
            parada['codigo_linha'] = i
            
        dados_paradas.append(parada_linha)

# Salvando dados de previsao de chegada por linha
agora = datetime.now().strftime("%Y-%m-%d %H%M%S")
path_paradas= f"raw\previsao\{agora}.json"
salvar_json(dados_paradas, path_paradas, 'utf-8')


# Finalizando a sessão quando não for mais necessária
session.close()

Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com s

# Renomear dados posicao

In [6]:
# Trazer nome dos campos com base no catálogo da api
df = pd.read_excel("catálogo api.xlsx", sheet_name="Objetos de retorno")

In [7]:
# Filtrar registros onde Categoria e Método = Posicao
df_filtrado = df[(df['Categoria'] == 'Posicao') & (df['Métodos'] == 'Posicao')]

# Criar o dicionário usando as colunas "Chave" e "Nome Campo"
mapa_chaves = dict(zip(df_filtrado['Chave'], df_filtrado['Nome Campo']))

# Aplicar função para ajustar nome dos dados
dados_renomeados = renomear_chaves(dados_posicao, mapa_chaves)

# Transformar o dicionário renomeado em um DataFrame
linhas_localizadas = dados_renomeados['linhas_localizadas']

# Extraindo informações dos veículos dentro de cada linha localizada
veiculos_lista = []
for linha in linhas_localizadas:
    for veiculo in linha['veiculos_localizados']:
        # Adicionando detalhes da linha junto com os veículos
        veiculo_completo = veiculo.copy()  # Copia as informações do veículo
        veiculo_completo.update({
            'hora_referencia_infos': dados_renomeados['hora_referencia_infos'],
            'letreiro_completo': linha['letreiro_completo'],
            'codigo_linha': linha['codigo_linha'],
            'sentido_linha': linha['sentido_linha'],
            'letreiro_destino_linha': linha['letreiro_destino_linha'],
            'letreiro_origem_linha': linha['letreiro_origem_linha'],
            'quantidade_veiculos_localizados': linha['quantidade_veiculos_localizados']
        })
        veiculos_lista.append(veiculo_completo)

# Criar o DataFrame com todas as informações dos veículos e suas respectivas linhas
df_completo = pd.DataFrame(veiculos_lista)


In [8]:
# Transformar em string e salvar o dataframe
df_completo = df_completo.astype(str)
#df_completo.to_csv('cleaned\posicao.csv', index=False, encoding='utf-8')
df_completo.to_excel('cleaned\posicao.xlsx', index=False)

# Renomear dados previsao de chegada por linha

In [9]:
# Filtrar registros onde Categoria e Método = Posicao
filtro_previsao = df[(df['Categoria'] == 'Previsao') & (df['Métodos'] == 'Linha')]

# Criar o dicionário usando as colunas "Chave" e "Nome Campo"
mapa_chaves_previsao = dict(zip(filtro_previsao['Chave'], filtro_previsao['Nome Campo']))

dados_previsao_renomeados = renomear_chaves(dados_previsao, mapa_chaves_previsao)


def json_previsao_para_df(dados_renomeados):
    veiculos_lista = []
    for previsao in dados_renomeados:
        codigo_linha = previsao['codigo_linha']  # Adiciona o código da linha
        for parada in previsao['relacao_pontos_parada']:
            for veiculo in parada['veiculos_localizados']:
                # Adiciona as informações da parada e da linha para cada veículo
                veiculo_completo = veiculo.copy()
                veiculo_completo.update({
                    'hora_referencia_infos': previsao['hora_referencia_infos'],
                    'codigo_parada': parada['codigo_parada'],
                    'nome_parada': parada['nome_parada'],
                    'latitude_localizada': parada['latitude_localizada'],
                    'longitude_localizada': parada['longitude_localizada'],
                    'codigo_linha': codigo_linha  # Inclui o código da linha
                })
                veiculos_lista.append(veiculo_completo)
    
    # Criar o DataFrame
    df_previsao = pd.DataFrame(veiculos_lista)
    return df_previsao

In [10]:
# Renomear os campos
df_previsao = json_previsao_para_df(dados_previsao_renomeados)

# Gerar o dataframe e salvar
df_previsao = df_previsao.astype(str)
#df_previsao.to_csv('cleaned\previsao.csv', index=False, encoding='utf-8')
df_previsao.to_excel('cleaned\previsao.xlsx', index=False)

# Renomear dados de paradas

In [11]:
# Filtrar registros onde Categoria e Método = Posicao
filtro_paradas = df[(df['Categoria'] == 'Parada') & (df['Métodos'] == 'BuscarParadasPorLinha')]

# Criar o dicionário usando as colunas "Chave" e "Nome Campo"
mapa_chaves_paradas = dict(zip(filtro_paradas['Chave'], filtro_paradas['Nome Campo']))

dados_paradas_renomeados = renomear_chaves(dados_paradas, mapa_chaves_paradas)

In [12]:
# Função para transformar o JSON renomeado em DataFrame
def json_paradas_para_df(dados_renomeados):
    paradas_lista = []
    
    # Percorre todas as sublistas no JSON
    for sublista in dados_renomeados:
        for parada in sublista:
            paradas_lista.append(parada)
    
    # Criar o DataFrame com os dados das paradas
    df_paradas = pd.DataFrame(paradas_lista)
    return df_paradas

# Transformar o JSON renomeado em DataFrame
df_paradas = json_paradas_para_df(dados_paradas_renomeados)


# Gerar o dataframe e salvar
df_paradas = df_paradas.astype(str)
#df_paradas.to_csv('cleaned\paradas.csv', index=False, encoding='utf-8')
df_paradas.to_excel('cleaned\paradas.xlsx', index=False)